<a id='table_of_contents'></a>

0. [Import libraries](#imports)
1. [Import data](#import_data)
2. [Initial Cleaning](#initial_cleaning)
3. [Price and Quantity Cleaning](#price_and_quantity_cleaning)
4. [Data Checks](#data_checks)
5. [Export cleaned data](#export_data)

# 0. Import libraries <a id='imports'></a>
[Back to top](#table_of_contents)

In [60]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import json

import pandas as pd
import numpy as np
from ipywidgets import VBox, HBox
import matplotlib.pyplot as plt
import matplotlib as mpl

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_colwidth", 100)
mpl.rcParams["figure.dpi"] = 300

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Import raw data <a id='import_data'></a>
[Back to top](#table_of_contents)

In [61]:
# Read in raw data
BASE_DIR = Path().resolve().parent
DATA_DIR = BASE_DIR / "data"

df_in = pd.read_csv(DATA_DIR / "raw" / "25072024_reviews.csv")
df_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7890 entries, 0 to 7889
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   rating                             7890 non-null   object 
 1   roaster                            7890 non-null   object 
 2   title                              7890 non-null   object 
 3   blind_assessment                   7889 non-null   object 
 4   notes                              7888 non-null   object 
 5   bottom_line                        3812 non-null   object 
 6   roaster location                   7887 non-null   object 
 7   coffee origin                      7386 non-null   object 
 8   roast level                        7488 non-null   object 
 9   agtron                             7890 non-null   object 
 10  est. price                         5852 non-null   object 
 11  review date                        7890 non-null   objec

# 2. Initial Cleaning <a id='initial_cleaning'></a>
[Back to top](#table_of_contents)

First step is to do some of the basic data checks and cleanup. This includes dropping columns that are not needed, setting datatypes, renaming columns,
combining columns, cleaning up strings, and creating new columns. 

In [62]:
# Cleanup column names
df_in.columns = (
    df_in.columns.str.strip().str.lower().str.replace(" ", "_").str.replace(".", "")
)

df_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7890 entries, 0 to 7889
Data columns (total 21 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   rating                            7890 non-null   object 
 1   roaster                           7890 non-null   object 
 2   title                             7890 non-null   object 
 3   blind_assessment                  7889 non-null   object 
 4   notes                             7888 non-null   object 
 5   bottom_line                       3812 non-null   object 
 6   roaster_location                  7887 non-null   object 
 7   coffee_origin                     7386 non-null   object 
 8   roast_level                       7488 non-null   object 
 9   agtron                            7890 non-null   object 
 10  est_price                         5852 non-null   object 
 11  review_date                       7890 non-null   object 
 12  aroma 

In [63]:
def tweak_df(df: pd.DataFrame) -> pd.DataFrame:
    """Initial data cleaning and feature creation"""
    return (
        df.assign(
            review_date=lambda df_: pd.to_datetime(df_["review_date"], format="%B %Y"),
            # Combing acidity and acidity/structure into one column, they are the same
            # field but names used in reviews changed at one point
            acidity=lambda df_: df_["acidity"].fillna(df_["acidity/structure"]),
            # Split the agtron column into one for external bean agtron data and ground
            # bean agtron data
            agtron_external=lambda df_: pd.to_numeric(
                df_["agtron"].str.split("/", expand=True)[0].str.strip(),
                errors="coerce",
            ),
            agtron_ground=lambda df_: pd.to_numeric(
                df_["agtron"].str.split("/", expand=True)[1].str.strip(),
                errors="coerce",
            ),
            # Distinguish espresso roasts from other reviews
            is_espresso=lambda df_: df_.apply(
                lambda row: (
                    True
                    if "espresso" in row["title"].lower()
                    or pd.notnull(row["with_milk"])
                    else False
                ),
                axis=1,
            ),
        )
        .replace(["", "NR", "N/A", "na"], np.nan)
        # Agtron values must be equalt to or below 100, some entries on website have typos
        .loc[
            lambda df_: (df_["agtron_external"] <= 100) & (df_["agtron_ground"] <= 100),
            :,
        ]
        # Run str.strip on every string column
        .map(lambda x: x.strip() if isinstance(x, str) else x)
        .drop(
            columns=["acidity/structure", "agtron", "refresh(enable_javascript_first)"]
        )
        .astype(
            {
                k: "float"
                for k in [
                    "agtron_external",
                    "agtron_ground",
                    "acidity",
                    "rating",
                    "aroma",
                    "body",
                    "flavor",
                    "aftertaste",
                ]
            }
        )
    )


df = tweak_df(df_in)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7566 entries, 0 to 7889
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   rating            7563 non-null   float64       
 1   roaster           7566 non-null   object        
 2   title             7566 non-null   object        
 3   blind_assessment  7566 non-null   object        
 4   notes             7565 non-null   object        
 5   bottom_line       3758 non-null   object        
 6   roaster_location  7564 non-null   object        
 7   coffee_origin     7299 non-null   object        
 8   roast_level       7445 non-null   object        
 9   est_price         5790 non-null   object        
 10  review_date       7566 non-null   datetime64[ns]
 11  aroma             7540 non-null   float64       
 12  acidity           6322 non-null   float64       
 13  body              7556 non-null   float64       
 14  flavor            7553 non-nu

# 3. Price and Quantity Cleaning <a id='price_and_quantity_cleaning'></a>
[Back to top](#table_of_contents)

The `est_price` column contains information on price, currency, and quantity. We need to split this column up to separate the price and quantity information.

Splitting on "/" character creates one column with price and currency information and another with quantity and unit of measurement information. 

The quantities have to be standardized so they contain a single representation for each unit and so unecessary punctuation and parentheses are removed.

We also filter the dataset to remove all products that came in units of cans, boxes, capusles, pods, etc. We will only concern ourselves with coffee sold in bags or bulk, ground or whole.

Regex is used to separate numerical and non-numerical characters from the quantity and price columns.


In [74]:
drop_terms = [
    "can",
    "box",
    "capsules",
    "K-",
    "cups",
    "bags",
    "concentrate",
    "discs",
    "bottle",
    "pods",
    "ml",
    "pods",
    "pouch",
    "packet|tin",
    "instant",
    "sachet",
    "vue",
    "single-serve",
    "fluid",
    "capsultes",
]

drop_terms_string = "|".join(drop_terms)


def price_quantity_split(df: pd.DataFrame) -> pd.DataFrame:
    print(f"Original df shape: {df.shape}")
    price_quantity = (
        df
        # Split est_price into columns for price and quantity
        .est_price.str.split("/", n=1, expand=True)
        # Remove any commas from the price and quantity columns
        .replace(",", "", regex=True)
        .rename(columns={0: "price", 1: "quantity"})
        .assign(
            # Cleanup quantity
            quantity=lambda df_: (
                df_["quantity"]
                .loc[
            lambda df_: ~df_["quantity"].str.contains(drop_terms_string, case=False),
            :,
        ]
                .str.lower().str.strip()
                # Remove parentheses and anything inside them
                .str.replace(r"\(.*?\)", "", regex=True)
                # Remove anything after a semicolon. This is usually a note, or deal price.
                .str.replace(r";.*", "", regex=True)
                # Standardize units
                .str.replace(r".g$", " grams", regex=True)
                .str.replace(r"\sg$", "grams", regex=True)
                .str.replace(r"pound$", "1 pounds", regex=True)
                .str.replace(r"oz|onces|ouncues|ounce$|ounces\*", "ounces", regex=True)
                .str.replace("\bkilo\b", "kilograms", regex=True)
                # Remove "online" from any quantity
                .str.replace("online", "")
                .str.strip()
            ),
            price=lambda df_: df_["price"].str.replace("..", "."),
        )
        .dropna()
        # Split quantity into value and unit, and split price into value and currency
        .assign(
            # Extract number value from quantity
            quantity_value=lambda df_: (
                df_["quantity"].str.extract(r"(\d+)").astype(float)
            ),
            # Extract the unit from quantity column
            quantity_unit=lambda df_: (
                df_["quantity"]
                .str.replace(r"(\d+)", "", regex=True)
                .replace("\.", "", regex=True)
                .str.strip()
                .mask(lambda s: s == "g", "grams")
                .str.strip()
            ),
            # Extract price value from price column
            price_value=lambda df_: (
                df_["price"].str.extract(r"(\d+\.\d+|\d+)").astype(float)
            ),
            # Extract currency from price column
            price_currency=lambda df_: (
                df_["price"]
                .str.replace(",", "")
                .str.replace(r"(\d+\.\d+|\d+)", "", regex=True)
                .str.strip()
            ),
        )
        # Drop the original price and quantity columns
        .drop(columns=["price", "quantity"])
    )
    print(f"Shape of price_quantity: {price_quantity.shape}")

    # Merge the price_quantity DataFrame with the original DataFrame
    return df.merge(price_quantity, how="left", left_index=True, right_index=True)


df = df_in.pipe(tweak_df).pipe(price_quantity_split)

df.quantity_unit.value_counts()

Original df shape: (7566, 21)
Shape of price_quantity: (5610, 4)


quantity_unit
ounces                         4757
grams                           823
pounds                           17
kg                                4
kilogram                          4
ounces (sold as a set of /$       3
kilo                              1
gram                              1
Name: count, dtype: int64

#### Cleaning Currencies

Normalize the currency column to contain a standardized set of currency symbols. We will use the ISO 4217 codes to make it easier to get foreign exchange data from an external API later on. 


In [57]:
def clean_currency(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize currencies to ISO 4217 codes."""
    price_currency = (
        df.price_currency.str.upper()
        .str.replace(r"^\$$", "USD", regex=True)
        .str.replace("PRICE: $", "USD")
        .str.replace("$", "")
        .str.replace("#", "GBP")
        .str.replace("¥", "JPY")
        .str.replace("£", "GBP")
        .str.replace("€", "EUR")
        .str.replace("POUND", "GBP")
        .str.replace("PESOS", "MXN")
        .str.replace("RMB", "CNY")
        .str.replace("EUROS", "EUR")
        .str.strip()
        .mask(lambda s: s == "US", "USD")
        .mask(lambda s: s == " ", "USD")
        .mask(lambda s: s == "E", "EUR")
        .mask(lambda s: s == "NTD", "TWD")
        .mask(lambda s: s == "NT", "TWD")
        .mask(lambda s: s == "", "USD")
        .mask(lambda s: s == "HK", "HKD")
        .str.strip()
    )
    return df.assign(price_currency=price_currency)


df = df_in.pipe(tweak_df).pipe(price_quantity_split).pipe(clean_currency)


# Check that currencies make sense from original est_price column
df.loc[:, ["est_price", "price_currency", "price_value"]].groupby(
    "price_currency"
).sample(3, replace=True)

Original df shape: (7566, 21)
Shape of price_quantity: (5786, 4)


,est_price,price_currency,price_value
7560,AED $99.75/250 grams,AED,99.75
7560,AED $99.75/250 grams,AED,99.75
5351,AED $103.95/250 grams,AED,103.95
1030,AUD $16.00/250 grams,AUD,16.00
7640,AUD $48.00/200 grams,AUD,48.00
...,...,...,...
3691,NT $750/8 ounces,TWD,750.00
5958,NT $698/100 grams,TWD,698.00
1404,$19.95/8 ounces,USD,19.95
394,$20.00/12 ounces,USD,20.00


In [43]:
df.price_currency.value_counts()

price_currency
USD    4267
TWD    1068
CAD     125
HKD      46
CNY      27
THB      21
KRW      20
JPY      12
GBP      10
AUD      10
EUR       6
AED       5
RM        3
IDR       3
GTQ       1
MXN       1
LAK       1
Name: count, dtype: int64

In [67]:
df.quantity_unit.value_counts()

quantity_unit
ounces                         4757
grams                           823
pounds                           17
kg                                4
ounces (sold as a set of /$       3
kilogram                          3
kilo                              1
gram                              1
Kilogram                          1
Name: count, dtype: int64

### Converting prices to 2024 USD

1. Convert price to USD using historical exchange rates
2. Adjust price to 2024 USD using BLS consumer price index

In [29]:
def convert_row(row):
    date = row["review_date"].strftime("%Y-%m-%d")
    currency = row["price_currency"]
    price = row["price_value"]
    if currency == "USD":
        return price
    else:
        return np.round(price / exchange_rates[date][currency], 2)


def convert_to_usd(df: pd.DataFrame) -> pd.DataFrame:
    df["price_value_usd_hist"] = df.apply(convert_row, axis=1)
    return df


# Read in exchange rates
with open(DATA_DIR / "external" / "openex_exchange_rates.json") as f:
    exchange_rates = json.load(f)

df = (
    df_in.pipe(tweak_df)
    .pipe(price_quantity_split)
    .pipe(clean_currency)
    .pipe(convert_to_usd)
)

(
    df.loc[:, ["price_value", "price_currency", "price_value_usd_hist"]].groupby(
        "price_currency"
    )
).sample(3, replace=True)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/tylernardone/Projects/coffee-review/data/external/openex_exchange_rates.json'

In [ ]:
def load_transform_cpi(file_path: Path) -> pd.DataFrame:
    """Loads and transforms the CPI data."""
    MONTH_MAP = {
        "Jan": 1,
        "Feb": 2,
        "Mar": 3,
        "Apr": 4,
        "May": 5,
        "Jun": 6,
        "Jul": 7,
        "Aug": 8,
        "Sep": 9,
        "Oct": 10,
        "Nov": 11,
        "Dec": 12,
    }

    try:
        cpi = pd.read_csv(
            file_path,
            usecols=[
                "Year",
                "Jan",
                "Feb",
                "Mar",
                "Apr",
                "May",
                "Jun",
                "Jul",
                "Aug",
                "Sep",
                "Oct",
                "Nov",
                "Dec",
            ],
        )
    except FileNotFoundError:
        raise FileNotFoundError("CPI file is not found in the specified directory.")

    return (
        cpi.melt(id_vars="Year", var_name="Month", value_name="cpi")
        .assign(
            Month=lambda x: x["Month"].map(MONTH_MAP),
            date=lambda x: pd.to_datetime(x[["Year", "Month"]].assign(day=1)),
        )
        .dropna()
        .drop(columns=["Year", "Month"])
        .rename(columns={"cpi": "consumer_price_index"})
        .sort_values("date")
        .reset_index(drop=True)
    )


def create_cpi_adjusted_price(
    df: pd.DataFrame, file_path: Path, date: str = "2024-01-01"
) -> pd.DataFrame:
    """
    Adjusts historical price data to 2024 prices using CPI data.
    """
    cpi = load_transform_cpi(file_path)
    cpi_baseline = cpi.loc[cpi["date"] == date, "consumer_price_index"].values[0]

    return (
        df.merge(cpi, left_on="review_date", right_on="date")
        .drop(columns="date")
        .assign(
            price_usd_adj_2024=lambda df_: np.round(
                df_["price_value_usd_hist"]
                * cpi_baseline
                / df_["consumer_price_index"],
                2,
            )
        )
    )


data_dir = Path("../../data")
cpi_path = data_dir / "external" / "consumer_price_index.csv"

df = (
    df_raw.pipe(tweak_df)
    .pipe(price_quantity_split)
    .pipe(clean_currency)
    .pipe(convert_to_usd)
    .pipe(create_cpi_adjusted_price, file_path=cpi_path)
)

df.sample(3)

In [ ]:
(
    df.assign(
        price_diff=lambda df_: (df_["price_usd_adj_2024"] - df_["price_value_usd_hist"])
        / df_["price_usd_adj_2024"]
    )
).plot(
    x="review_date",
    y="price_diff",
    title="% Price difference between adjusted and historical prices",
)

### Converting quantities to lbs

Create a normalized quantity column that converts all quantities to lbs.

In [ ]:
def convert_to_lbs(df: pd.DataFrame) -> pd.DataFrame:
    to_lbs_conversion = {
        "ounces": 1 / 16,
        "pounds": 1,
        "kilogram": 2.20462,
        "grams": 0.00220462,
    }
    df["quantity_in_lbs"] = np.round(
        df["quantity_value"] * df["quantity_unit"].map(to_lbs_conversion), 2
    )
    return df


df = (
    df_raw.pipe(tweak_df)
    .pipe(price_quantity_split)
    .pipe(clean_currency)
    .pipe(convert_to_usd)
    .pipe(create_cpi_adjusted_price, file_path=cpi_path)
    .pipe(convert_to_lbs)
)

(
    df.loc[:, ["quantity_value", "quantity_unit", "quantity_in_lbs"]]
    .groupby("quantity_unit")
    .sample(3, replace=True)
)

In [ ]:
# Create a new column for price per pound
def price_per_lbs(df: pd.DataFrame) -> pd.DataFrame:
    df["price_usd_adj_2024_per_lb"] = np.round(
        df["price_usd_adj_2024"] / df["quantity_in_lbs"], 2
    )
    return df


df = (
    df_raw.pipe(tweak_df)
    .pipe(price_quantity_split)
    .pipe(clean_currency)
    .pipe(convert_to_usd)
    .pipe(create_cpi_adjusted_price, file_path=cpi_path)
    .pipe(convert_to_lbs)
    .pipe(price_per_lbs)
)

df.describe()

In [ ]:
# list of US states
us_states_and_territories = [
    "Alabama",
    "Alaska",
    "Arizona",
    "Arkansas",
    "California",
    "Colorado",
    "Connecticut",
    "Delaware",
    "Florida",
    "Georgia",
    "Hawaii",
    "Idaho",
    "Illinois",
    "Indiana",
    "Iowa",
    "Kansas",
    "Kentucky",
    "Louisiana",
    "Maine",
    "Maryland",
    "Massachusetts",
    "Michigan",
    "Minnesota",
    "Mississippi",
    "Missouri",
    "Montana",
    "Nebraska",
    "Nevada",
    "New Hampshire",
    "New Jersey",
    "New Mexico",
    "New York",
    "North Carolina",
    "North Dakota",
    "Ohio",
    "Oklahoma",
    "Oregon",
    "Pennsylvania",
    "Rhode Island",
    "South Carolina",
    "South Dakota",
    "Tennessee",
    "Texas",
    "Utah",
    "Vermont",
    "Virginia",
    "Washington",
    "West Virginia",
    "Wisconsin",
    "Wyoming",
    "District of Columbia",
    "Puerto Rico",
]


def create_us_state(row):
    if row["territorial_entity_2"] in us_states_and_territories:
        return row["territorial_entity_2"]
    elif row["territorial_entity_1"] in us_states_and_territories:
        return row["territorial_entity_1"]
    elif row["og_roaster_location"].split(",")[-1].strip() in us_states_and_territories:
        return row["og_roaster_location"].split(",")[-1].strip()
    else:
        return np.nan


def create_county_and_state_columns(df: pd.DataFrame) -> pd.DataFrame:
    df["roaster_county"] = np.where(
        df["territorial_entity_1"].str.contains("County", na=False),
        df["territorial_entity_1"],
        np.nan,
    )
    df["roaster_us_state"] = df.apply(create_us_state, axis=1)
    return df


df = (
    df_raw.pipe(tweak_df)
    .pipe(price_quantity_split)
    .pipe(clean_currency)
    .pipe(convert_to_usd)
    .pipe(create_cpi_adjusted_price, file_path=cpi_path)
    .pipe(convert_to_lbs)
    .pipe(price_per_lbs)
    .pipe(create_county_and_state_columns)
)

display(
    df.loc[
        df["roaster_country"] == "USA",
        ["roaster_country", "roaster_us_state", "roaster_county"],
    ].info()
)

# 4. Data Checks <a id='data_checks'></a>
[Back to top](#table_of_contents)

In [ ]:
df = (
    df_raw.pipe(tweak_df)
    .pipe(price_quantity_split)
    .pipe(clean_currency)
    .pipe(convert_to_usd)
    .pipe(create_cpi_adjusted_price, file_path=cpi_path)
    .pipe(convert_to_lbs)
    .pipe(price_per_lbs)
    .pipe(create_county_and_state_columns)
)

df.info()

In [ ]:
df_numeric = df.select_dtypes(include=["number"]).drop(columns=["price_value"], axis=1)
len(df_numeric.columns)

In [ ]:
fig, ax = plt.subplots(3, 5, figsize=(15, 10))

for i, col in enumerate(df_numeric.columns):
    df[col].plot(
        kind="hist",
        ax=ax[i // 5, i % 5],
        title=col,
        bins=20,
        edgecolor="black",
        alpha=0.7,
    )

plt.tight_layout()
plt.show()

In [ ]:
df[df["price_usd_adj_2024_per_lb"] < 200].price_usd_adj_2024_per_lb.hist(
    cumulative=True, density=True, edgecolor="black", alpha=0.7
)
plt.title("Cumulative Histogram Price $USD/lbs")

In [ ]:
# Checking countries
display(df.roaster_country.sort_values().unique())
display(df.coffee_origin_country.sort_values().unique())

In [ ]:
# Checking roast level
df.roast_level.value_counts()

# 5. Export cleaned data <a id='export_data'></a>
[Back to top](#table_of_contents)

In [ ]:
fout = data_dir / "processed" / "05052024_roast_review_cleaned.csv"
df.to_csv(fout, index=False)